In [7]:
import pandas as pd
import tabula
import numpy as np

tables=tabula.read_pdf("axis05.pdf",
                  pages='all',
                  silent=True,
                  multiple_tables=True,
                  pandas_options={'header':None})

df = pd.DataFrame()
df = pd.concat([c for c in tables]).drop_duplicates()

try:
    idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
    df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)
except:
    try:
        idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
        df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)           
    except:
        print("\nAxis-Column headers missing"); log.write("\nAxis-Column headers missing"); pass
    
try:
    df = df.drop(["Init."], axis=1)
except:
    try:
        df = df.drop(["Branch Name"], axis=1)
    except:pass
df.head()

df=df[~df.index.isin(df[df.apply(lambda row: row.astype(str).str.lower().str.contains('opening balance|transaction total|closing balance').any(), axis=1) ==True].index)]
df.drop(df.nunique(dropna=False)[(df.nunique(dropna=False) == 1)].index, axis=1,inplace=True)

try:
    narr=[n for n in df.columns if "PARTICULARS" in str(n).upper()][0]
except:
    try:
        narr=[n for n in df.columns if "NARRATION" in str(n).upper()][0]
    except:
        try:
            narr=[n for n in df.columns if "DESCRIPTION" in str(n).upper()][0] 
        except:
            pass

df = df[pd.notnull(df[narr])]

df.iloc[:,-1]=df.iloc[:,-1].fillna(df.iloc[:,-2])

def abc(a):
    if type(a) ==str:
        if len(a.split(' '))==2:
            z=a.split(' ')[1]
        else:
            z=a.split(' ')[0]
    else :
        z=a
    return z

try:
    bal=[c for c in df.columns if "BALANCE" in str(c).upper()][0]
    df[bal]=df[bal].apply(lambda x: abc(x))
except:
    print("\nBalance Column Missing")
    
df["flag"]=df.iloc[:,-1].shift(1)
df=df[~df.index.isin(df[df.iloc[:,-1]==df.iloc[:,-2]].index)]

df=df.T.drop_duplicates().T
df[bal]=df[bal].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)
df["flag"]=df.iloc[:,0].astype(str)+df[bal].astype(str)
df["flag2"]=np.arange(len(df))

df.loc[df[["flag"]].duplicated(keep=False), 'flag'] = df["flag"]+df["flag2"].astype(str)
df['flag']=df['flag'].apply(lambda row : np.nan if 'nannan' in row else row).fillna(method='bfill')

df[narr]=df.groupby('flag')[narr].transform(lambda x:''.join(x))
df=df.drop_duplicates(['flag'],keep='last').iloc[0:,:-2].reset_index(drop=True)
df.to_csv("parsed.csv",index=0)
print("Completed..........")

KeyError: '[nan] not found in axis'